In [1]:
import pandas as pd

In [2]:
amazon = pd.read_csv('/content/amazon_clean_headlines.csv')
apple = pd.read_csv('/content/apple_clean_headlines.csv')
google = pd.read_csv('/content/google_clean_headlines.csv')
microsoft = pd.read_csv('/content/microsoft_clean_headlines.csv')
nvidia = pd.read_csv('/content/nvidia_clean_headlines.csv')

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [5]:
amazon_headlines = amazon['cleaned_headline'].tolist()
apple_headlines = apple['cleaned_headline'].tolist()
google_headlines = google['cleaned_headline'].tolist()
microsoft_headlines = microsoft['cleaned_headline'].tolist()
nvidia_headlines = nvidia['cleaned_headline'].tolist()

In [7]:
import torch
company_dfs = [amazon, apple, google, microsoft, nvidia]
company_names = ['amazon', 'apple', 'google', 'microsoft', 'nvidia']
results = {}

for df, name in zip(company_dfs, company_names):
    headlines = df['cleaned_headline'].tolist()
    inputs = tokenizer(headlines, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    results[name] = outputs

In [8]:
import torch.nn.functional as F

probabilities = {}

for name, output in results.items():
    probs = F.softmax(output.logits, dim=-1)
    probabilities[name] = probs

In [9]:
labels = ['positive', 'negative', 'neutral']

for df, probs in zip(company_dfs, probabilities.values()):
    predicted_classes = probs.argmax(axis=1)
    df['sentiment'] = [labels[i] for i in predicted_classes]
    df['sentiment_score'] = probs.max(axis=1).values

In [13]:
company_dfs = [amazon, apple, google, microsoft, nvidia]
company_names = ['amazon', 'apple', 'google', 'microsoft', 'nvidia']

for df, name in zip(company_dfs, company_names):
    filename = f"{name}-final-cleaned-headlines.csv"
    df.to_csv(filename, index=False)